## Baseline

Below is the baseline benchmark without shaping.

In [1]:
!iperf3 -c 172.17.0.2 -p 8080 -t 10

Connecting to host 172.17.0.2, port 8080
[  5] local 172.17.0.3 port 43290 connected to 172.17.0.2 port 8080
[ ID] Interval           Transfer     Bitrate         Retr  Cwnd
[  5]   0.00-1.00   sec  1.28 GBytes  11.0 Gbits/sec    0    639 KBytes       
[  5]   1.00-2.00   sec  1.80 GBytes  15.5 Gbits/sec    0    639 KBytes       
[  5]   2.00-3.00   sec  1.81 GBytes  15.6 Gbits/sec    0    639 KBytes       
[  5]   3.00-4.00   sec  1.87 GBytes  16.1 Gbits/sec    0    639 KBytes       
[  5]   4.00-5.00   sec  1.83 GBytes  15.7 Gbits/sec    0    639 KBytes       
[  5]   5.00-6.00   sec  1.87 GBytes  16.1 Gbits/sec    0    639 KBytes       
[  5]   6.00-7.00   sec  1.80 GBytes  15.5 Gbits/sec    0    639 KBytes       
[  5]   7.00-8.00   sec  1.92 GBytes  16.5 Gbits/sec    0    639 KBytes       
[  5]   8.00-9.00   sec  1.83 GBytes  15.7 Gbits/sec    0    639 KBytes       
[  5]   9.00-10.00  sec  1.87 GBytes  16.1 Gbits/sec    0    639 KBytes       
- - - - - - - - - - - - - - - - - - 

## Shaping

Now we create the `qdisc`, `class` and `filter` as shown in <https://youtu.be/Ylf4J736JIg>.

In [2]:
!tc qdisc add dev eth0 root handle 1:0 htb default 30
!tc qdisc show dev eth0

!tc class add dev eth0 parent 1:0 classid 1:1 htb rate 100mbit
!tc class show dev eth0

!tc filter add dev eth0 protocol ip parent 1:0 prio 1 u32 match ip dst 172.16.0.0/12 flowid 1:1
!tc filter show dev eth0

qdisc htb 1: root refcnt 2 r2q 10 default 0x30 direct_packets_stat 0 direct_qlen 1000
class htb 1:1 root prio 0 rate 100Mbit ceil 100Mbit burst 1600b cburst 1600b 
filter parent 1: protocol ip pref 1 u32 chain 0 
filter parent 1: protocol ip pref 1 u32 chain 0 fh 800: ht divisor 1 
filter parent 1: protocol ip pref 1 u32 chain 0 fh 800::800 order 2048 key ht 800 bkt 0 flowid 1:1 not_in_hw 
  match ac100000/fff00000 at 16


In [3]:
!iperf3 -c 172.17.0.2 -p 8080 -t 10

Connecting to host 172.17.0.2, port 8080
[  5] local 172.17.0.3 port 34226 connected to 172.17.0.2 port 8080
[ ID] Interval           Transfer     Bitrate         Retr  Cwnd
[  5]   0.00-1.00   sec  14.9 MBytes   125 Mbits/sec    0   1.50 MBytes       
[  5]   1.00-2.00   sec  11.2 MBytes  94.4 Mbits/sec    0   1.50 MBytes       
[  5]   2.00-3.00   sec  12.5 MBytes   105 Mbits/sec    0   1.50 MBytes       
[  5]   3.00-4.00   sec  11.2 MBytes  94.4 Mbits/sec    0   1.50 MBytes       
[  5]   4.00-5.00   sec  11.2 MBytes  94.4 Mbits/sec    0   1.50 MBytes       
[  5]   5.00-6.00   sec  12.5 MBytes   105 Mbits/sec    0   1.50 MBytes       
[  5]   6.00-7.00   sec  11.2 MBytes  94.4 Mbits/sec    0   1.50 MBytes       
[  5]   7.00-8.00   sec  11.2 MBytes  94.4 Mbits/sec    0   1.50 MBytes       
[  5]   8.00-9.00   sec  12.5 MBytes   105 Mbits/sec    0   1.50 MBytes       
[  5]   9.00-10.00  sec  11.2 MBytes  94.3 Mbits/sec    0   1.50 MBytes       
- - - - - - - - - - - - - - - - - - 

After an initial burst to 125 Mbits/sec, the bandwidth is limited to 100 Mbits/sec as configured on the class `1:1`.

## Cleanup

In [4]:
!tc qdisc del dev eth0 root

Once the `root` qdisc has been deleted, the bandwidth goes back to the baseline.

In [5]:
!iperf3 -c 172.17.0.2 -p 8080 -t 10

Connecting to host 172.17.0.2, port 8080
[  5] local 172.17.0.3 port 38344 connected to 172.17.0.2 port 8080
[ ID] Interval           Transfer     Bitrate         Retr  Cwnd
[  5]   0.00-1.00   sec  1.67 GBytes  14.4 Gbits/sec    0    639 KBytes       
[  5]   1.00-2.00   sec  1.96 GBytes  16.8 Gbits/sec    0    639 KBytes       
[  5]   2.00-3.00   sec  1.82 GBytes  15.6 Gbits/sec    0    639 KBytes       
[  5]   3.00-4.00   sec  1.49 GBytes  12.8 Gbits/sec    0    639 KBytes       
[  5]   4.00-5.00   sec  1.85 GBytes  15.9 Gbits/sec    0    639 KBytes       
[  5]   5.00-6.00   sec  1.89 GBytes  16.2 Gbits/sec    0    639 KBytes       
[  5]   6.00-7.00   sec  1.71 GBytes  14.7 Gbits/sec    0    639 KBytes       
[  5]   7.00-8.00   sec  1.92 GBytes  16.5 Gbits/sec    0    639 KBytes       
[  5]   8.00-9.00   sec  1.84 GBytes  15.8 Gbits/sec    0    639 KBytes       
[  5]   9.00-10.00  sec  1.96 GBytes  16.8 Gbits/sec    0    639 KBytes       
- - - - - - - - - - - - - - - - - - 